In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
plt.close("all") 
# DÚVIDAS NO COMANDO ACIMA?
# https://matplotlib.org/2.1.0/api/_as_gen/matplotlib.pyplot.close.html

In [ ]:
dados = pd.read_csv("/kaggle/input/avocado-prices/avocado.csv")

In [ ]:
dados.head()

In [ ]:
print(dados["Total Volume"].max())

In [ ]:
dados.info()

In [ ]:
dados["type"]

In [ ]:
tipos_abacates = dados["type"].unique().tolist()
# DÚVIDAS NO COMANDO ACIMA?
# https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html
print("Existem " + str(len(tipos_abacates)) + " tipos de abacates nos dados:")
for tipo in tipos_abacates:
    print("\t- " + tipo)

In [ ]:
dados["region"]

In [ ]:
regioes = dados["region"].unique().tolist()
# DÚVIDAS NO COMANDO ACIMA?
# https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html
print("Existem " + str(len(regioes)) + " regiões distintas:")
for regiao in regioes:
    print("\t- " + regiao)

<p>
    Bom.
    <br />
    Agora que entendo um poucos os dados, já começam brotar as primeiras curiosidades: 
</p>
<ol>
    <li>
        Como será a relação de preços pelo tipo?
    </li>
    <li>
        Como será a relação de preços pela época do ano?
    </li>
    <li>
        Existe uma relação de migração de consumo sazonal, considerando as regiões?
    </li>
    <li>
        Existe uma relação de consumo com o preço?
    </li>
</ol>

<i>
    Acabei de ver que não sei o significado dos campos abaixo:
    <br />
    <ul>
        <li>
            Total Volume
        </li>
        <li>
            4046
        </li>
        <li>
            4225
        </li>
        <li>
            4770
        </li>
        <li>
            Total Bags
        </li>
        <li>
            Small Bags
        </li>
        <li>
            Large Bags
        </li>
        <li>
            XLarge Bags
        </li>
    </ul>
    <br />
    Vou parar e fazer minhas pesquisas. Não conheço o negógio de venda de abacates.
</i>

In [ ]:
total_de_linhas = len(dados.index)

In [ ]:
dados["Bags"] = dados["Small Bags"] + dados["Large Bags"] + dados["XLarge Bags"]

dados_filtrados_Bags_PrimeiroCaso = dados[(
    dados["Total Bags"] == dados["Bags"] 
)]

dados["Bagsm"] = dados["Bags"].multiply(0.9999)
dados["BagsM"] = dados["Bags"].multiply(1.0001)
# DÚVIDAS NO COMANDO ACIMA?
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.multiply.html
dados_filtrados_Bags_SegundoCaso = dados[(
    (dados["Total Bags"] >= dados["Bagsm"]) 
    &
    (dados["Total Bags"] <= dados["BagsM"])
)]


print(
    str(len(dados_filtrados_Bags_PrimeiroCaso.index)) 
    + 
    " de um total de " 
    + 
    str(total_de_linhas)
)

print(
    str(len(dados_filtrados_Bags_SegundoCaso.index)) 
    + 
    " de um total de " 
    +  
    str(total_de_linhas)
)

<p>
    Dá pra dizer que Total Bags, com uma variação de 0.0002, é a soma de Small Bags, Large Bags e XLarge Bags, apesar de ainda não saber o que esses nomes significam. Não sei ainda ao certo a quantidade de abacates em cada tipo de sacola dessas. 
</p>
<p>
    Vou correlacionar graficamente os números para tentar visualizar essa relação. 
</p>

In [ ]:
dados.plot.scatter(x="Total Bags", y="Bags");

In [ ]:
dados["Volumes"] = dados["4046"] + dados["4225"] + dados["4770"]

dados_filtrados_Volumes_PrimeiroCaso = dados[(
    dados["Total Volume"] == dados["Volumes"] 
)]

dados["Volumesm"] = dados["Volumes"].multiply(0.7)
dados["VolumesM"] = dados["Volumes"].multiply(1.3)
# DÚVIDAS NO COMANDO ACIMA?
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.multiply.html
dados_filtrados_Volumes_SegundoCaso = dados[(
    (dados["Total Volume"] >= dados["Volumesm"]) 
    &
    (dados["Total Volume"] <= dados["VolumesM"])
)]


print(
    str(len(dados_filtrados_Volumes_PrimeiroCaso.index)) 
    + 
    " de um total de " 
    + 
    str(total_de_linhas)
)

print(
    str(len(dados_filtrados_Volumes_SegundoCaso.index)) 
    + 
    " de um total de " 
    +  
    str(total_de_linhas)
)

In [ ]:
dados.plot.scatter(x="Total Volume", y="Volumes");

<p>
    Não estou vendo relação <b>AINDA</b> entre "Total Volume" e a soma das parcelas 4046, 4225 e 4770. Vou precisar entender melhor o significado desses campos. 
</p>

In [ ]:
dados["DiferencaVolume"] = dados["Total Volume"] - dados["Volumes"]
print(dados["DiferencaVolume"].min())
print(dados["DiferencaVolume"].max())

In [ ]:
plt.figure(figsize = (13, 5))
tamanho_histograma = int(len(dados["DiferencaVolume"].unique().tolist()) / 1000)
dados["DiferencaVolume"].plot.hist(bins = tamanho_histograma, log = True)
# DÚVIDAS NO COMANDO ACIMA?
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html

In [ ]:
dados["Total Volume Acumulado"] = dados["Total Volume"].cumsum()
dados["Volumes Acumulado"] = dados["Volumes"].cumsum()
# DÚVIDAS NO COMANDO ACIMA?
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumsum.html
dados.plot(x = "Date", y = ["Total Volume Acumulado", "Volumes Acumulado"], figsize = (15, 5));

In [ ]:
dados.head()

<p>
    Encontrei as duas fontes abaixo que explicam o negócio de frutas:
    <ul>
        <li>
            <a href = "https://cran.r-project.org/web/packages/avocado/vignettes/a_intro.html" target = "_blank">Fonte 1</a>
        </li>
        <li>
            <a href = "https://en.wikipedia.org/wiki/Price_look-up_code" target = "_blank">Fonte 2</a>
        </li>
    </ul>
</p>


In [ ]:
dados.boxplot(column="AveragePrice", by="region", figsize = (15, 5), rot = 90)

<span style = "font-size: 24px; color: #ff0000;">
    Até aqui "Versão de Conhecimento dos Dados".
</span>